In [1]:
import sys  
sys.path.insert(0, '/home/cbisot/pycode/MscThesis/')
%matplotlib widget
from util import get_path, get_dates_datetime
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton, from_connection_tab, from_nx_to_tab
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign
from datetime import datetime,timedelta
from node_id import orient
import pickle
from matplotlib.widgets import CheckButtons
import scipy.io as sio
import imageio
from pymatreader import read_mat
from matplotlib import colors
from copy import deepcopy,copy
from collections import Counter
import cv2 
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi, meijering
from skimage.morphology import thin
from skimage import data, filters
from random import choice
import scipy.sparse
import os
from time import time
from extract_graph import dic_to_sparse, from_sparse_to_graph, generate_nx_graph, prune_graph, from_nx_to_tab, from_nx_to_tab_matlab,sparse_to_doc, connections_pixel_list_to_tab, transform_list, clean_degree_4
from time import sleep
from skimage.feature import hessian_matrix_det
from experiment_class_surf import Experiment,clean_exp_with_hyphaes
from hyphae_id_surf import clean_and_relabel, get_mother, save_hyphaes, resolve_ambiguity_two_ends,solve_degree4, clean_obvious_fake_tips, get_pixel_growth_and_new_children
from subprocess import call
import open3d as o3d
import sklearn
from Analysis.util import get_time, get_length_um
from experiment_class_surf import Edge,Node

In [23]:
from directory import directory

# exp_inst = [(40,0,37),(13,0,19),(3,0,19)]
exp_inst = [(13,0,19)]

experiments = []
for inst in exp_inst:
    plate = inst[0]
    begin = inst[1]
    end = inst[2]
    dates_datetime = get_dates_datetime(directory,plate)
    print('begin =',dates_datetime[begin],'\n  end =',dates_datetime[end])
    dates_datetime_chosen=dates_datetime[begin:end+1]
    dates = dates_datetime_chosen
    exp = pickle.load( open(f'{directory}/Analysis_Plate{plate}_{dates[0]}_{dates[-1]}/experiment_{plate}.pick', "rb" ) )
    experiments.append(exp)

begin = 2020-07-01 15:57:00 
  end = 2020-07-04 19:57:00


In [24]:
exp = experiments[0]
# exp13 = experiments[1]
# exp3 = experiments[2]

In [25]:
def get_length_um_edge(edge,t):
    pixel_conversion_factor = 1.725
    length_edge = 0
    pixels = edge.pixel_list(t)
    for i in range(len(pixels) // 10 + 1):
        if i * 10 <= len(pixels) - 1:
            length_edge += np.linalg.norm(
                np.array(pixels[i * 10])
                - np.array(pixels[min((i + 1) * 10, len(pixels) - 1)])
            )
    #         length_edge+=np.linalg.norm(np.array(pixels[len(pixels)//10-1*10-1])-np.array(pixels[-1]))
    return length_edge * pixel_conversion_factor

In [26]:
def get_length_um_node_list(node_list,exp,t):
    total_length = 0
    for i in range(len(node_list)-1):
        nodea=Node(node_list[i],exp)
        nodeb=Node(node_list[i+1],exp)
        edge_obj =Edge(nodea,nodeb,exp)
        total_length += get_length_um_edge(edge_obj,t)
    return(total_length)
        

In [27]:
def estimate_bas_freq(exp,samples,t0,thresh_growth=150,thresh_length=1000):
    select_hyph = {}
    for hyph in exp.hyphaes:
        select_hyph[hyph] = []
        for i,t in enumerate(hyph.ts[:-1]):
            tp1=hyph.ts[i+1]
            pixels,nodes = get_pixel_growth_and_new_children(hyph,t,tp1)
            length = np.sum([get_length_um(seg) for seg in pixels])/get_time(exp,t,tp1)
            select_hyph[hyph].append((t,hyph.ts[i+1],length,pixels,nodes))
    max_growths = []
    lengths = []
    branch_frequ = []
    hyph_l = []
    RH = []
    BAS = []
    hyph_info = {}
    for hyph in exp.hyphaes:
        growths = [c[2] for c in select_hyph[hyph]]
        ts = [c[0] for c in select_hyph[hyph]]
        tp1s = [c[1] for c in select_hyph[hyph]]
        if len(growths)>0:
            length = hyph.get_length_um(hyph.ts[-1])
            nodes = hyph.get_nodes_within(hyph.ts[-1])[0]
            max_growth = np.max(growths)
            total_growth = np.sum([growth * get_time(exp,ts[i],tp1s[i]) for i,growth in enumerate(growths)])
            if max_growth>=thresh_growth  and length>=thresh_length:
                RH.append(hyph)
            else:
                BAS.append(hyph)
            lengths.append(length)
            max_growths.append(max_growth)
            branch_frequ.append((len(nodes)-1)/(length+1))
            hyph_l.append(hyph)
            hyph_info[hyph]={'max_growth':max_growth,'length_total':length,'total_growth':total_growth,'nodes':nodes}
    bas_frequ=[]
    bas_roots = [hyph.root for hyph in BAS]
    for k in range(samples):
        node1 = Node(choice(list(exp.nx_graph[t0].nodes)),exp)
        node2 = Node(choice(list(exp.nx_graph[t0].nodes)),exp)
        if np.linalg.norm(node1.pos(t0)-node2.pos(t0))>=5000:
            nodes = nx.shortest_path(exp.nx_graph[t0], source = node1.label, target = node2.label)
#             exp.plot([t],[nodes])
            bass=[]
            for node in nodes:
                if exp.get_node(node) in  bas_roots:
                    bass.append(hyph)
        #             print(hyph)
        #             if hyph.ts[0] in rh.ts:
        #                 print(hyph,hyph_info[hyph],hyph in BAS)
        #                 hyph.end.show_source_image(hyph.ts[-1],hyph.ts[-1])
            bas_frequ.append(len(bass)/get_length_um_node_list(nodes,exp,t0)*10000)
    return(bas_frequ)

In [28]:
bas_frequ = estimate_bas_freq(exp,200,19,thresh_growth=100,thresh_length=2000)

In [29]:
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.set_xlabel('bas_frequ (bas/mm)')
ax.set_ylabel('density')
# ax.set_xlim((0,2))
ax.hist(np.array(bas_frequ),20,density=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([0.01459043, 0.02918086, 0.05836171, 0.04377129, 0.27721814,
        0.204266  , 0.43771286, 0.37935114, 0.39394157, 0.21885643,
        0.11672343, 0.08754257, 0.07295214, 0.102133  , 0.02918086,
        0.04377129, 0.02918086, 0.01459043, 0.        , 0.02918086]),
 array([ 5.72524924,  6.11247004,  6.49969085,  6.88691166,  7.27413246,
         7.66135327,  8.04857408,  8.43579488,  8.82301569,  9.2102365 ,
         9.5974573 ,  9.98467811, 10.37189892, 10.75911972, 11.14634053,
        11.53356133, 11.92078214, 12.30800295, 12.69522375, 13.08244456,
        13.46966537]),
 <a list of 20 Patch objects>)

In [21]:
np.mean(bas_frequ),np.std(bas_frequ)

(8.881609370900788, 1.7020854242027799)

In [ ]:
bas_frequ = estimate_bas_freq(exp13,200,19,thresh_growth=100,thresh_length=2000)

In [ ]:
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.set_xlabel('bas_frequ (bas/mm)')
ax.set_ylabel('density')
# ax.set_xlim((0,2))
ax.hist(np.array(bas_frequ),10,density=True)

In [ ]:
np.mean(bas_frequ),np.std(bas_frequ)

In [ ]:
plt.close('all')
bas_frequ = estimate_bas_freq(exp3,200,19,thresh_growth=100,thresh_length=2000)

In [ ]:
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.set_xlabel('bas_frequ (bas/mm)')
ax.set_ylabel('density')
# ax.set_xlim((0,2))
ax.hist(np.array(bas_frequ),10,density=True)

In [ ]:
np.mean(bas_frequ),np.std(bas_frequ)

In [ ]:
exp = exp40
select_hyph = {}
for hyph in exp.hyphaes:
    select_hyph[hyph] = []
    for i,t in enumerate(hyph.ts[:-1]):
        tp1=hyph.ts[i+1]
        pixels,nodes = get_pixel_growth_and_new_children(hyph,t,tp1)
        length = np.sum([get_length_um(seg) for seg in pixels])/get_time(exp,t,tp1)
        select_hyph[hyph].append((t,hyph.ts[i+1],length,pixels,nodes))

In [12]:
max_growths = []
lengths = []
branch_frequ = []
hyph_l = []
RH=[]
BAS=[]
thresh_growth = 150
thresh_length = 1000
hyph_info = {}
for hyph in exp.hyphaes:
    growths = [c[2] for c in select_hyph[hyph]]
    ts = [c[0] for c in select_hyph[hyph]]
    tp1s = [c[1] for c in select_hyph[hyph]]
    if len(growths)>0:
        length = hyph.get_length_um(hyph.ts[-1])
        nodes = hyph.get_nodes_within(hyph.ts[-1])[0]
        max_growth = np.max(growths)
        total_growth = np.sum([growth * get_time(exp,ts[i],tp1s[i]) for i,growth in enumerate(growths)])
        if max_growth>=thresh_growth  and length>=thresh_length:
            RH.append(hyph)
        else:
            BAS.append(hyph)
        lengths.append(length)
        max_growths.append(max_growth)
        branch_frequ.append((len(nodes)-1)/(length+1))
        hyph_l.append(hyph)
        hyph_info[hyph]={'max_growth':max_growth,'length_total':length,'total_growth':total_growth,'nodes':nodes}

In [65]:
last_t = len(exp.nx_graph)-1
node1 = Node(choice(list(exp.nx_graph[last_t].nodes)),exp)
node2 = Node(choice(list(exp.nx_graph[last_t].nodes)),exp)

In [ ]:
plt.close('all')
nodes = nx.shortest_path(exp.nx_graph[-1], source = node1.label, target = node2.label)
exp.plot([last_t],[nodes])

In [67]:
last_t = len(exp.nx_graph)-1
bass=[]
for node in nodes:
    for hyph in exp.hyphaes:
        if exp.get_node(node)==hyph.root:
            if hyph in BAS:
                bass.append(hyph)
                break
#             print(hyph)
#             if hyph.ts[0] in rh.ts:
#                 print(hyph,hyph_info[hyph],hyph in BAS)
#                 hyph.end.show_source_image(hyph.ts[-1],hyph.ts[-1])
len(bass)/get_length_um_node_list(nodes,exp,last_t)*10000


9.18428587716088

In [ ]:
bas_frequ = estimate_bas_freq(exp,samples,t,thresh_growth=150,thresh_length=1000)

In [ ]:
edge = choice(list(exp.nx_graph[-1].edges))

In [21]:
for bas in BAS:
    result = bas.get_nodes_within(bas.ts[-1])
    break

In [ ]:
edge

In [56]:
bas_frequ=[]
for rh in RH:
    bass=[]
    for node in rh.get_nodes_within(rh.ts[-1])[0][1:-1]:
        for hyph in exp.hyphaes:
            if exp.get_node(node)==hyph.root:
                if hyph in BAS:
                    bass.append(hyph)
                    break
    #             print(hyph)
    #             if hyph.ts[0] in rh.ts:
#                 print(hyph,hyph_info[hyph],hyph in BAS)
#                 hyph.end.show_source_image(hyph.ts[-1],hyph.ts[-1])
    bas_frequ.append(len(bass)/hyph_info[rh]['length_total'])
            

In [58]:
bas_frequ

[0.00017338764807121762,
 4.953932802034789e-05,
 4.953932802034789e-05,
 4.953932802034789e-05,
 4.953932802034789e-05,
 4.953932802034789e-05,
 7.430899203052184e-05,
 4.953932802034789e-05,
 2.4769664010173946e-05,
 2.4769664010173946e-05,
 4.953932802034789e-05,
 9.907865604069578e-05,
 0.0005697022722340007,
 2.4769664010173946e-05,
 2.4769664010173946e-05,
 4.953932802034789e-05,
 2.4769664010173946e-05,
 4.953932802034789e-05,
 0.0,
 4.953932802034789e-05,
 9.907865604069578e-05,
 9.907865604069578e-05,
 7.430899203052184e-05,
 9.907865604069578e-05,
 0.0,
 9.907865604069578e-05,
 4.953932802034789e-05,
 2.4769664010173946e-05,
 7.430899203052184e-05,
 9.907865604069578e-05,
 2.4769664010173946e-05,
 0.0,
 0.0,
 2.4769664010173946e-05,
 4.953932802034789e-05,
 2.4769664010173946e-05,
 2.4769664010173946e-05,
 0.00017338764807121762,
 4.953932802034789e-05,
 0.00017338764807121762,
 9.907865604069578e-05,
 4.953932802034789e-05,
 2.4769664010173946e-05,
 0.00017338764807121762,
 

In [ ]:
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.set_xlabel('bas_frequ (bas/mm)')
ax.set_ylabel('density')
# ax.set_xlim((0,2))
ax.hist(np.array(bas_frequ)*10000,10,density=True)

In [8]:
len(RH)

126

In [174]:
rh = choice(RH)
rh.ts

[27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37]

In [175]:
plt.close('all')
t0 = select_hyph[rh][0][0]
times=[get_time(exp,t0,c[0]) for c in select_hyph[rh]]
speeds = [c[2] for c in select_hyph[rh]]
plt.close('all')
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('time (hour)')
ax.set_ylabel('speed (um/hour)')
ax.plot(times,speeds)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [176]:
rh.get_nodes_within(rh.ts[-1])[0]

[9110, 9873, 9865, 9855, 9109]

In [177]:
plt.close('all')
exp.plot(rh.ts,[rh.get_nodes_within(rh.ts[-1])[0]+[]]*len(rh.ts))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [178]:
plt.close('all')
for node in rh.get_nodes_within(rh.ts[-1])[0][1:-1]:
    for hyph in exp.hyphaes:
        if exp.get_node(node)==hyph.root:
#             print(hyph)
#             if hyph.ts[0] in rh.ts:
            print(hyph,hyph_info[hyph],hyph in BAS)
            hyph.end.show_source_image(hyph.ts[-1],hyph.ts[-1])

(Node(9862), Node(9873)) (68.55772085948638, 432.47828936436974, 148.6429469009527, [9873, 14704, 9862]) True


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(Node(11846), Node(9865)) (0.0, 139.72161256677649, 0.0, [9865, 11165, 11846]) True


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
np.mean(bas_frequ)*10000,np.std(bas_frequ)*10000

(7.646535511603008, 5.143899884825599)

In [25]:
np.mean(bas_frequ)*10000,np.std(bas_frequ)*10000

(8.98550841574617, 4.786839847744506)

In [9]:
np.mean(bas_frequ)*10000,np.std(bas_frequ)*10000

(4.84338132339359, 3.625468207834553)

In [2]:
15*4096*1.725

105984.0

In [92]:
plt.close('all')
exp.plot(rh.ts,[[rh.end.label,rh.root.label]]*len(rh.ts))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [160]:
rh.end.ts()

[16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]

In [159]:
rh.end.show_source_image(rh.ts[-1],rh.ts[-1]+1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [40]:
edge

9823

In [45]:
edge = choice(list(exp.nx_graph[-1].edges))
edge_obj =Edge(Node(edge[0],exp),Node(edge[1],exp),exp)

In [49]:
edge[0]

998